<h3 style="text-align: center; font-family: Arial, sans-serif; color: #4CAF50;">Dictionnaire des Datasources</h3>
<table style="width: 100%; border-collapse: collapse; font-family: Arial, sans-serif; font-size: 12pt; text-align: left;">
    <thead>
        <tr style="background-color: #f2f2f2;">
            <th style="border: 1px solid #ddd; padding: 8px; color: #4CAF50; text-align: center; width: 30%;">Nom du Fichier</th>
            <th style="border: 1px solid #ddd; padding: 8px; color: #4CAF50; text-align: center; width: 20%;">Nom du DataFrame</th>
            <th style="border: 1px solid #ddd; padding: 8px; color: #4CAF50; text-align: center; width: 50%;">Description</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="border: 1px solid #ddd; padding: 1px; text-align: left;">MetroPT3.csv</td>
            <td style="border: 1px solid #ddd; padding: 1px; text-align: center;">df</td>
            <td style="border: 1px solid #ddd; padding: 1px; text-align: left;">Dataset Original</td>
        </tr>
        <tr>
            <td style="border: 1px solid #ddd; padding: 1px; text-align: left;">MetroPT3_corrected.csv</td>
            <td style="border: 1px solid #ddd; padding: 1px; text-align: center;">df</td>
            <td style="border: 1px solid #ddd; padding: 1px; text-align: left;">Dataset avec des observations vides, timestamp avec des gaps de 10 sec respectés avec tag de la panne (windows 15mn)</td>
        </tr>
         <tr>
            <td style="border: 1px solid #ddd; padding: 1px; text-align: left;">MetroPT3_corrected_marked.csv</td>
            <td style="border: 1px solid #ddd; padding: 1px; text-align: center;">df</td>
            <td style="border: 1px solid #ddd; padding: 1px; text-align: left;">Dataset avec des observations vides, timestamp avec des gaps de 10 sec respectés et missed colonnes ajoutées</td>
        </tr>
    </tbody>
</table>


In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [2]:
# Charger les données
df = pd.read_csv(r"..\..\..\Datasources\MetroPT3.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)

In [3]:
display(df.head(2))

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
0,2020-02-01 00:00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
1,2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.04,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [3]:
# Convertir timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
print(df.dtypes)

timestamp          datetime64[ns]
TP2                       float64
TP3                       float64
H1                        float64
DV_pressure               float64
Reservoirs                float64
Oil_temperature           float64
Motor_current             float64
COMP                      float64
DV_eletric                float64
Towers                    float64
MPG                       float64
LPS                       float64
Pressure_switch           float64
Oil_level                 float64
Caudal_impulses           float64
dtype: object


In [5]:
# Filtrer uniquement les colonnes numériques
numeric_columns = df.select_dtypes(include=[np.number])

In [6]:
##################################################################
##### Ajout des TimeStamp manquants en 2 étapes              #####
##################################################################

#####################
##### Étape 1 : #####
#####################

# Fonction d'ajustement du dernier chiffre des secondes
def adjust_last_digit_of_seconds(timestamp):
    """Ajuste uniquement le dernier chiffre des secondes du timestamp."""
    seconds = timestamp.second
    last_digit = seconds % 10  # Extraire le dernier chiffre
    
    if 1 <= last_digit <= 5:
        # Arrondir à 0
        adjustment = -last_digit
    elif last_digit > 5:
        # Arrondir au multiple de 10 suivant
        adjustment = 10 - last_digit
    else:
        # Si le dernier chiffre est déjà 0
        adjustment = 0

    # Appliquer l'ajustement aux secondes
    return timestamp + pd.Timedelta(seconds=adjustment)

# Appliquer la correction à tous les timestamps avec une barre de progression
tqdm.pandas(desc="Ajustement des timestamps")
df['timestamp'] = df['timestamp'].progress_apply(adjust_last_digit_of_seconds)

# Vérification des résultats
print("Aperçu des timestamps corrigés :")
print(df['timestamp'].head())


Ajustement des timestamps: 100%|██████████████████████████████████████████| 1516948/1516948 [00:45<00:00, 33663.76it/s]

Aperçu des timestamps corrigés :
0   2020-02-01 00:00:00
1   2020-02-01 00:00:10
2   2020-02-01 00:00:20
3   2020-02-01 00:00:30
4   2020-02-01 00:00:40
Name: timestamp, dtype: datetime64[ns]


In [14]:
#####################
##### Étape 2 : #####
#####################

# Démarrer le chronomètre
start_time = time.time()

# Recalculer dynamiquement les intervalles après chaque correction
while True:
    # Calculer les intervalles
    time_diffs = df['timestamp'].diff().dt.total_seconds()

    # Identifier les intervalles problématiques (tout ce qui n'est pas 10 secondes)
    problematic_indices = time_diffs[time_diffs > 10].index

    if len(problematic_indices) == 0:
        print("Toutes les irrégularités ont été corrigées.")
        break

    # Cibler la première irrégularité détectée
    idx = problematic_indices[0]

    # Récupérer le timestamp de départ
    start_time = df.loc[idx - 1, 'timestamp']
    end_time = df.loc[idx, 'timestamp']

    # Calculer les timestamps intermédiaires pour combler l'intervalle
    new_times = pd.date_range(start=start_time, end=end_time, freq='10S', closed='right')

    # Créer un DataFrame avec les nouveaux timestamps
    missing_data = pd.DataFrame({'timestamp': new_times})

    # Ajouter les nouveaux timestamps dans le DataFrame
    df = pd.concat([df, missing_data]).drop_duplicates(subset='timestamp').sort_values('timestamp').reset_index(drop=True)

    # Afficher l'état actuel
    print(f"Correction appliquée à l'intervalle autour de l'index {idx}.")
    print(df.loc[max(idx - 2, 0):min(idx + len(new_times) + 2, len(df) - 1), ['timestamp']])

# Vérification finale
time_diffs_corrected = df['timestamp'].diff().dt.total_seconds()
print("\nStatistiques des intervalles corrigés après correction finale :")
print(f"Min : {time_diffs_corrected.min()} secondes")
print(f"Max : {time_diffs_corrected.max()} secondes")
print(f"Mode (valeur typique) : {time_diffs_corrected.mode()[0]} secondes")
print(f"Nombre d'irrégularités restantes : {(time_diffs_corrected != 10).sum()}")


# Arrêter le chronomètre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")


Toutes les irrégularités ont été corrigées.

Statistiques des intervalles corrigés après correction finale :
Min : 10.0 secondes
Max : 10.0 secondes
Mode (valeur typique) : 10.0 secondes
Nombre d'irrégularités restantes : 1
Temps d'exécution: 0.00 secondes


In [26]:
##################################################################
##### Ajout de la variable Panne                             #####
##################################################################

# Ajouter les classes de panne
# Classe 0 : Pas de panne détectée
# Classe 1 : En pleine panne
# Classe 2 : Panne prévue dans moins de 15 minutes

# Liste des intervalles
# Datset : start: 2020-02-01 00:00:00 --> end: 2020-09-01 03:59:50
pannes = [
    {'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
    {'start': '2020-05-29 23:30:00', 'end': '2020-05-30 06:00:00'},
    {'start': '2020-06-05 10:00:00', 'end': '2020-06-07 14:30:00'},
    {'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
         ]

# Convertir les timestamps des pannes en datetime
for panne in pannes:
    panne['start'] = pd.to_datetime(panne['start'])
    panne['end']   = pd.to_datetime(panne['end'])

# Ajouter une colonne 'panne' avec la valeur par défaut 0 (aucune panne détectée)
df['panne'] = 0


for panne in pannes:
    # Classe 1 : En pleine panne
    df.loc[(df['timestamp'] >= panne['start']) & (df['timestamp'] <= panne['end']), 'panne'] = 1
    # Classe 2 : Panne prévue dans moins de 15 minutes
    df.loc[(df['timestamp'] < panne['start']) & (df['timestamp'] >= panne['start'] - pd.Timedelta(minutes=15)), 'panne'] = 2

display(df.head(2))

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,panne
0,2020-02-01 00:00:00,-0.01,9.36,9.34,-0.02,9.36,53.60,0.04,1.00,0.00,1.00,1.00,0.00,1.00,1.00,1.00,0
1,2020-02-01 00:00:10,-0.01,9.35,9.33,-0.02,9.35,53.68,0.04,1.00,0.00,1.00,1.00,0.00,1.00,1.00,1.00,0


In [23]:
# Vérification des GAPS

# Calculer la différence entre chaque timestamp consécutif
df['timestamp'] = pd.to_datetime(df['timestamp'])
time_diffs = df['timestamp'].diff().dt.total_seconds().dropna()

# Vérifier si tous les intervalles sont égaux à 10 secondes
if (time_diffs == 10).all():
    print("Tous les intervalles sont de 10 secondes.")
else:
    print("Des écarts irréguliers ont été détectés.")
    print(time_diffs.value_counts())

# Affichage des premieres différences pour validation
print(time_diffs.head(10))

Tous les intervalles sont de 10 secondes.
1    10.00
2    10.00
3    10.00
4    10.00
5    10.00
6    10.00
7    10.00
8    10.00
9    10.00
10   10.00
Name: timestamp, dtype: float64


In [21]:
# Enregistrer le dataframe corrigé dans un fichier CSV
df.to_csv(r"..\..\..\Datasources\MetroPT3_corrected.csv", index=True)

In [27]:
#########################################################################################
##### Marquer le dataset avec des colonnes ajoutés pour taguer les valeurs imputées #####
#########################################################################################

# Ajouter une colonne "_is_missing" pour chaque colonne existante
for col in df.columns:
    if col not in ['timestamp', 'panne']:      # Exclure 'timestamp' et 'panne'
        df[f"{col}_is_missing"] = df[col].isnull().astype(int)

# Calculer le nombre de `1` dans chaque colonne `_is_missing`
missing_counts = {col: df[col].sum() for col in df.columns if col.endswith('_is_missing')}

# Convertir les résultats en DataFrame pour un affichage clair
missing_counts_df = pd.DataFrame(list(missing_counts.items()), columns=['Colonne', 'Nombre de 1'])

# Afficher les résultats
print("\nNombre de `1` dans chaque colonne `_is_missing` :")
display(missing_counts_df)


Nombre de `1` dans chaque colonne `_is_missing` :


,Colonne,Nombre de 1
0,TP2_is_missing,337637
1,TP3_is_missing,337637
2,H1_is_missing,337637
3,DV_pressure_is_missing,337637
4,Reservoirs_is_missing,337637
5,Oil_temperature_is_missing,337637
6,Motor_current_is_missing,337637
7,COMP_is_missing,337637
8,DV_eletric_is_missing,337637
9,Towers_is_missing,337637


In [28]:
# Enregistrer le dataframe corrigé dans un fichier CSV
df.to_csv(r"..\..\..\Datasources\MetroPT3_corrected_marked.csv", index=True)

In [ ]:
##################################################################
##### MODIFICATION DFANS LE TAG DU DATASET                   #####
##################################################################

In [2]:
# Charger les données
df = pd.read_csv(r"..\..\..\Datasources\MetroPT3_corrected_marked.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)

In [4]:
df.drop(columns=['panne'], inplace=True)

In [5]:
display(df.head(2))

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,...,Oil_temperature_is_missing,Motor_current_is_missing,COMP_is_missing,DV_eletric_is_missing,Towers_is_missing,MPG_is_missing,LPS_is_missing,Pressure_switch_is_missing,Oil_level_is_missing,Caudal_impulses_is_missing
0,2020-02-01 00:00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.04,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.04,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
##################################################################
##### Ajout de la variable Panne                             #####
##################################################################

# Ajouter les classes de panne
# Classe 0 : Pas de panne détectée
# Classe 1 : En pleine panne
# Classe 2 : Panne prévue dans moins de 15 minutes

# Liste des intervalles
# Datset : start: 2020-02-01 00:00:00 --> end: 2020-09-01 03:59:50
# pannes = [
#     {'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
#     {'start': '2020-05-29 23:30:00', 'end': '2020-05-30 06:00:00'},
#     {'start': '2020-06-05 10:00:00', 'end': '2020-06-07 14:30:00'},
#     {'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
#          ]

# Dataset commence le 2020-04-12 11:20:00 et se termine le 2020-07-17 06:00:00
pannes = [
    {'id': 'Panne1',  'start': '2020-04-12 11:50:00', 'end': '2020-04-12 23:30:00'},
    {'id': 'Panne2',  'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
    {'id': 'Panne3',  'start': '2020-04-19 00:00:00', 'end': '2020-04-19 01:30:00'},
    {'id': 'Panne4',  'start': '2020-04-29 03:20:00', 'end': '2020-04-29 04:00:00'},
    {'id': 'Panne5',  'start': '2020-04-29 22:00:00', 'end': '2020-04-29 22:20:00'},
    {'id': 'Panne6',  'start': '2020-05-13 14:00:00', 'end': '2020-05-13 23:59:00'},
    {'id': 'Panne7',  'start': '2020-05-18 05:00:00', 'end': '2020-05-18 05:30:00'},
    {'id': 'Panne8',  'start': '2020-05-19 10:10:00', 'end': '2020-05-19 11:00:00'},
    {'id': 'Panne9',  'start': '2020-05-19 22:10:00', 'end': '2020-05-19 23:59:00'},
    {'id': 'Panne10', 'start': '2020-05-20 00:00:00', 'end': '2020-05-20 20:00:00'},
    {'id': 'Panne11', 'start': '2020-05-23 09:50:00', 'end': '2020-05-23 10:10:00'},
    {'id': 'Panne12', 'start': '2020-05-29 23:30:00', 'end': '2020-05-29 23:59:00'},
    {'id': 'Panne13', 'start': '2020-05-30 00:00:00', 'end': '2020-05-30 06:00:00'},
    {'id': 'Panne14', 'start': '2020-06-01 15:00:00', 'end': '2020-06-01 15:40:00'},
    {'id': 'Panne15', 'start': '2020-06-03 10:00:00', 'end': '2020-06-03 11:00:00'},
    {'id': 'Panne16', 'start': '2020-06-05 10:00:00', 'end': '2020-06-05 23:59:00'},
    {'id': 'Panne17', 'start': '2020-06-06 00:00:00', 'end': '2020-06-06 23:59:00'},
    {'id': 'Panne18', 'start': '2020-06-07 00:00:00', 'end': '2020-06-07 14:30:00'},
    {'id': 'Panne19', 'start': '2020-07-08 17:30:00', 'end': '2020-07-08 19:00:00'},
    {'id': 'Panne20', 'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
    {'id': 'Panne21', 'start': '2020-07-17 04:30:00', 'end': '2020-07-17 05:30:00'}
         ]

# Hypothse : Le dataset pris en consideration sera 30 mn avant la premiere panne jusque 30 mn apres la derniere panne

df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Convertir les timestamps des pannes en datetime
for panne in pannes:
    panne['start'] = pd.to_datetime(panne['start'])
    panne['end']   = pd.to_datetime(panne['end'])

# Ajouter une colonne 'panne' avec la valeur par défaut 0 (aucune panne détectée)
df['panne'] = 0


for panne in pannes:
    # Classe 1 : En pleine panne
    df.loc[(df['timestamp'] >= panne['start']) & (df['timestamp'] <= panne['end']), 'panne'] = 1
    # Classe 2 : Panne prévue dans moins de 15 minutes
    df.loc[(df['timestamp'] < panne['start']) & (df['timestamp'] >= panne['start'] - pd.Timedelta(minutes=15)), 'panne'] = 2

display(df.head(2))

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,...,Motor_current_is_missing,COMP_is_missing,DV_eletric_is_missing,Towers_is_missing,MPG_is_missing,LPS_is_missing,Pressure_switch_is_missing,Oil_level_is_missing,Caudal_impulses_is_missing,panne
0,2020-02-01 00:00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.04,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.04,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Identifier le timestamp 30 minutes avant la première panne pour definir le debut du dataset 
start_time = min([panne['start'] for panne in pannes]) - pd.Timedelta(minutes=30)

# Identifier le timestamp 30 minutes après la dernière panne
end_time = max([panne['end'] for panne in pannes]) + pd.Timedelta(minutes=30)

# Filtrer le DataFrame pour ne garder que cette plage de temps
df_filtered = df[(df['timestamp'] >= start_time) & (df['timestamp'] <= end_time)]

# Afficher les premières lignes du DataFrame final
display(df_filtered.head(2))

# Facultatif : afficher les dimensions du DataFrame filtré pour validation
print(f"Dimensions du DataFrame filtré : {df_filtered.shape}")


,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,...,Motor_current_is_missing,COMP_is_missing,DV_eletric_is_missing,Towers_is_missing,MPG_is_missing,LPS_is_missing,Pressure_switch_is_missing,Oil_level_is_missing,Caudal_impulses_is_missing,panne
617520,2020-04-12 11:20:00,9.128,8.650,-0.018,-0.022,8.648,55.475,6.0175,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
617521,2020-04-12 11:20:10,9.354,8.896,-0.016,-0.020,8.892,56.525,6.0075,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0


Dimensions du DataFrame filtré : (827521, 32)


In [12]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_filtered[['timestamp','panne']].head(181))

,timestamp,panne
617520,2020-04-12 11:20:00,0
617521,2020-04-12 11:20:10,0
617522,2020-04-12 11:20:20,0
617523,2020-04-12 11:20:30,0
617524,2020-04-12 11:20:40,0
617525,2020-04-12 11:20:50,0
617526,2020-04-12 11:21:00,0
617527,2020-04-12 11:21:10,0
617528,2020-04-12 11:21:20,0
617529,2020-04-12 11:21:30,0


In [13]:
# Enregistrer le dataframe corrigé dans un fichier CSV
df_filtered.to_csv(r"..\..\..\Datasources\MetroPT3_corrected_new_marked.csv", index=True)